In [2]:
import sqlite3
from ossapi import Ossapi
from time import strftime, localtime
import pandas as pd
from gensim.test.utils import datapath
from gensim import utils
import gensim
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.model_selection import train_test_split
import os
from sklearn.neighbors import NearestNeighbors

import numpy as np
import sys

sys.path.insert(0, '../') # Add parent directory because data is in parent directory

In [4]:
# df = pd.read_parquet('../data/top_sentences_std.parquet', engine='pyarrow')
# sentences = df['sentences'].tolist()
df = pd.read_parquet('../data/hiscore_updates.parquet', engine='pyarrow')


In [6]:
len(df)

56995129

In [24]:
class MonitorCallback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(
        self, model
    ):  # word2vec accumulates loss, so we need to subtract the previous loss
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print("Loss after epoch {}: {}".format(self.epoch, loss))
        else:
            print(
                "Loss after epoch {}: {}".format(
                    self.epoch, loss - self.loss_previous_step
                )
            )
        self.epoch += 1
        self.loss_previous_step = loss

model = gensim.models.Word2Vec(
    sentences=sentences,
    vector_size=15,
    epochs=70,
    window=30,
    min_count=5,  
    workers=16,
    sg=0,
    hs=0, 
    negative=20,
    ns_exponent=0.9, 
    compute_loss=True,
    callbacks=[MonitorCallback()],
)

# loss = model.get_latest_training_loss()

Loss after epoch 0: 8154777.0
Loss after epoch 1: 6797529.0
Loss after epoch 2: 6145848.0
Loss after epoch 3: 5913650.0
Loss after epoch 4: 6006714.0
Loss after epoch 5: 4759366.0
Loss after epoch 6: 4690960.0
Loss after epoch 7: 4766720.0
Loss after epoch 8: 4765056.0
Loss after epoch 9: 4776596.0
Loss after epoch 10: 4844796.0
Loss after epoch 11: 4826476.0
Loss after epoch 12: 1819424.0
Loss after epoch 13: 1364048.0
Loss after epoch 14: 1381712.0
Loss after epoch 15: 1360128.0
Loss after epoch 16: 1346272.0
Loss after epoch 17: 1353440.0
Loss after epoch 18: 1363736.0
Loss after epoch 19: 1373080.0
Loss after epoch 20: 1371032.0
Loss after epoch 21: 1355536.0
Loss after epoch 22: 1368648.0
Loss after epoch 23: 1358768.0
Loss after epoch 24: 1355504.0
Loss after epoch 25: 1336384.0
Loss after epoch 26: 1312016.0
Loss after epoch 27: 1316920.0
Loss after epoch 28: 1326504.0
Loss after epoch 29: 1317408.0
Loss after epoch 30: 1317184.0
Loss after epoch 31: 1297480.0
Loss after epoch 3

In [25]:
model.save('word2vec_3.model')

In [15]:
# # word2vec_model = model
# # else:
#     # word2vec_model = gensim.models.Word2Vec.load("word2vec-pp/word2vec-pp")
# model = gensim.models.Word2Vec.load("w2v_model/w2v_model")
# model.wv.index_to_key[10000:20000]
word2vec_model = model
NN = NearestNeighbors(n_neighbors=50, algorithm='ball_tree').fit(word2vec_model.wv.vectors)

In [16]:
import sys
sys.path.insert(0, "../")
from data.classes import Score

OSU_CLIENT_ID = os.environ.get("OSU_CLIENT_ID")
OSU_CLIENT_SECRET = os.environ.get("OSU_CLIENT_SECRET")
user_id = '28956125'

api = Ossapi(OSU_CLIENT_ID, OSU_CLIENT_SECRET)
top_scores = api.user_scores(user_id, type="best", mode="osu", limit=100)

top_scores = [Score(score) for score in top_scores]
top_scores.sort(key=lambda x: x.pp, reverse=True)

# top_scores = [
#     str(score.beatmap_id) + "-" + str(score.mods) for score in top_scores # Limit to top 50
# ]

top_scores = ['1872812-0']
top_scores_vec = [
    score for score in top_scores if score in word2vec_model.wv.index_to_key
]
top_scores_vec = [word2vec_model.wv[score] for score in top_scores_vec]

neighbor = NN.kneighbors([np.mean(top_scores_vec, axis=0)])

In [17]:
# Get the top 5 beatmaps
beatmaps = [model.wv.index_to_key[i] for i in neighbor[1][0]]
beatmaps = [beatmap for beatmap in beatmaps if beatmap not in top_scores]

beatmaps

['845391-0',
 '2477065-0',
 '1637465-0',
 '1750943-0',
 '620789-0',
 '3018109-0',
 '635679-0',
 '1093605-0',
 '370440-0',
 '442170-0',
 '1557471-0',
 '1744286-0',
 '771858-0',
 '1045757-0',
 '931860-0',
 '386728-0',
 '2615748-0',
 '1188172-0',
 '1959500-0',
 '1502678-0',
 '2023961-0',
 '1972802-0',
 '655300-0',
 '2012740-0',
 '855536-0',
 '2077120-0',
 '2333933-0',
 '1868547-0',
 '2826252-0',
 '2357806-0',
 '767774-0',
 '221777-0',
 '2619918-0',
 '589969-0',
 '381928-0',
 '1863228-0',
 '3211514-0',
 '3385965-0',
 '209276-0',
 '2403266-0',
 '693195-0',
 '2779000-0',
 '2920787-0',
 '2015472-0',
 '2309955-0',
 '720590-0',
 '1601387-0',
 '1432512-0',
 '793845-0']